# Zero-Noise Extrapolation: the N=2 case

Authors: Kyle Godbey, Alexandra Semposki

Maintainer: Alexandra Semposki

Here we'll try developing some error mitigation for the $N=2$ deuteron case from the VQE chapter. Let's begin by importing <tt>pennylane</tt>, <tt>mitiq</tt>, and any other packages we may need, and copy over the code from the $N=2$ notebook for calculating the ground state energy.

In [11]:
%matplotlib inline

import matplotlib.pyplot as plt
from pennylane import numpy as np
import pennylane as qml
import warnings
warnings.filterwarnings('ignore')

import mitiq as mq
from mitiq.zne.scaling import fold_global
from mitiq.zne.inference import RichardsonFactory
from pennylane.transforms import mitigate_with_zne
from qiskit.providers.aer import AerSimulator
import qiskit.providers.aer.noise as noise

## VQE for N=2 (with noisy simulator)

Here we'll pull from some of the VQE notebook to define the Hamiltonian and circuit again. If you would like to revisit the VQE process, go to [this page](https://kylegodbey.github.io/nuclear-quantum-computing/vqe/n2.html). 

Next, we define a simulator that has a noise model imbedded into it, so that we are simulating running on a real quantum device. Let's see how the convergence of the VQE routine goes.

In [22]:
#define the Hamiltonian we're using again, for N=2
coeffs = [5.906709, 0.218291, -6.125, -2.143304, -2.143304]
obs = [qml.Identity(0), qml.PauliZ(0), qml.PauliZ(1), qml.PauliX(0) @ qml.PauliX(1), qml.PauliY(0) @ qml.PauliY(1)]

H = qml.Hamiltonian(coeffs, obs)

print(H)

  (-6.125) [Z1]
+ (0.218291) [Z0]
+ (5.906709) [I0]
+ (-2.143304) [X0 X1]
+ (-2.143304) [Y0 Y1]


In [46]:
#define some noise model to use in pennylane
prob_1 = 0.001; prob_2 = 0.01 #for one, two qubit gates
error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

#define the simulator to use
dev_sim = qml.device('qiskit.aer', wires=2, optimization_limit=0, backend='aer_simulator', noise_model=noise_model)

#create Qnodes to use
@qml.transforms.mitigate_with_zne([1, 2, 3], fold_global, RichardsonFactory.extrapolate)
@qml.qnode(dev_sim)

#now we'll define the circuit we want to use with this Hamiltonian
def circuit(params, wires):
    t0 = params[0]
    qml.PauliX(wires=0)
    qml.RY(t0, wires=1)
    qml.CNOT(wires=[1,0])

#define cost function
cost_fn = qml.ExpvalCost(circuit, H, dev_sim)

#set up the Qnode to run the above circuit on the simulator (is this necessary?)
sim_qnode = qml.QNode(circuit, dev_sim)

Now we define some initial parameter and convergence info, as before.

In [50]:
#parameter array
init_params = np.array([2.5,])

#convergence information and step size
max_iterations = 500
conv_tol = 1e-06
step_size = 0.01

Let's run VQE as in the VQE chapter. We should get a result that takes longer to converge due to the noise we have added into the device.

In [48]:
#VQE step
opt = qml.GradientDescentOptimizer(stepsize=step_size)

params = init_params

gd_param_history = [params]
gd_cost_history = []

for n in range(max_iterations):

    # Take a step in parameter space and record your energy
    params, prev_energy = opt.step_and_cost(cost_fn, params)

    # This keeps track of our energy for plotting at comparisons
    gd_param_history.append(params)
    gd_cost_history.append(prev_energy)

    # Here we see what the energy of our system is with the new parameters
    energy = cost_fn(params)

    # Calculate difference between new and old energies
    conv = np.abs(energy - prev_energy)

    if n % 20 == 0:
        print(
            "Iteration = {:},  Energy = {:.8f} MeV,  Convergence parameter = {"
            ":.8f} MeV".format(n, energy, conv)
        )

    if conv <= conv_tol:
        break

print()
print("Final value of the energy = {:.8f} MeV".format(energy))
print("Number of iterations = ", n)

QuantumFunctionError: A quantum function must return either a single measurement, or a nonempty sequence of measurements.